In [1]:
"""
Algorithm flow

while not converged:
    take s1[idx of positive example], s2[C1*len(s2) negative example], target[1...1,0...0]
    get the user's item_vec from model and extract s1, s2 part as model's output
    calculate loss (BCEWithLogitsLoss)
    update model
    

ISSUE: 
1. Batch?
2. Validation set?

"""

"\nAlgorithm flow\n\nwhile not converged:\n    take s1[idx of positive example], s2[C1*len(s2) negative example], target[1...1,0...0]\n    get the user's item_vec from model and extract s1, s2 part as model's output\n    calculate loss (BCEWithLogitsLoss)\n    update model\n    \n\nISSUE: \n1. Batch?\n2. Validation set?\n\n"

In [2]:
"""
Params
"""

train_file = "../dataset/train.csv"


In [3]:
"""Use gpu"""
import torch
print(torch.cuda.is_available())
torch.cuda.set_device(0)
torch.set_default_tensor_type('torch.cuda.FloatTensor')

True


In [3]:
"""
PIR_Dataset
"""
import torch
from torch.utils.data import Dataset
import pandas as pd
from random import sample, shuffle

C1 = 1

class PIR_Dataset(Dataset):

    def __init__(self, csv_file, valid_split=0.1):
        user_dataframe = pd.read_csv(csv_file)
        self.train_lists, self.valid_lists = self._get_pos_lists(user_dataframe, valid_split)
        self.num_items = max([max(l) for l in self.train_lists])+1
        self.num_users = len(self.train_lists)
        self.neg_lists = self._get_neg_lists()
        self.train_targets, self.valid_targets = self._get_targets()
        self.is_train = True
        
    def set_is_train(self, is_train):
        self.is_train = is_train
    
    def __len__(self):
        return len(self.train_lists)

    def __getitem__(self, idx):
        if self.is_train == True:
            s1 = self.train_lists[idx]
            s2 = sample(self.neg_lists[idx], C1*len(self.train_lists[idx]))
            return s1, s2, self.train_targets[idx]
        else:
            # might sample neg_data from validation set but ignore for now
            s1 = self.valid_lists[idx]
            s2 = sample(self.neg_lists[idx], C1*len(self.valid_lists[idx]))
            return s1, s2, self.valid_targets[idx]
    
    def _get_pos_lists(self, df, valid_split):
        tot_lists = [list(map(int, item_str.split())) for item_str in df["ItemId"]]
        train_lists, valid_lists = [], []
        for l in tot_lists:
            shuffle(l)
            idx = int(len(l)*valid_split) + 1
            valid_lists.append(sorted(l[:idx]))
            train_lists.append(sorted(l[idx:]))
        return train_lists, valid_lists
    
    def _get_neg_lists(self):
        neg_lists = [[] for _ in range(self.num_users)]
        for user_id in range(self.num_users):
            pos_iter = 0
            for item_id in range(self.num_items):
                if item_id == self.train_lists[user_id][pos_iter]:
                    pos_iter += (pos_iter != len(self.train_lists[user_id])-1)
                else:
                    neg_lists[user_id].append(item_id)
        return neg_lists
    
    def _get_targets(self):
        return [torch.cat((torch.ones(len(l)), torch.zeros(C1*len(l)))) for l in self.train_lists], \
                [torch.cat((torch.ones(len(l)), torch.zeros(C1*len(l)))) for l in self.valid_lists]
    

In [4]:
"""
PIR_Model
"""
import torch

NUM_TOPICS = 256

class PIR_Model(torch.nn.Module):
    
    def __init__(self, num_users, num_items, topics=NUM_TOPICS):
        super().__init__()
        self.user_matrix = torch.randn((num_users, topics), requires_grad=True)
        self.item_matrix = torch.randn((topics, num_items), requires_grad=True)
        self.item_bias = torch.randn((num_items,), requires_grad=True)
        
    def forward(self, user_id, pos_idx, neg_idx):
        pos_tensor = torch.matmul(self.user_matrix[user_id], self.item_matrix[:,pos_idx]) + self.item_bias[pos_idx]
        neg_tensor = torch.matmul(self.user_matrix[user_id], self.item_matrix[:,neg_idx]) + self.item_bias[neg_idx]
        if pos_tensor.dim() == 1:
            return torch.cat((pos_tensor, neg_tensor))
        else:
            return torch.cat((pos_tensor, neg_tensor), axis=1)
    

In [5]:
"""
PIR_Loss
"""



class BPR_Loss(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.m = torch.nn.LogSigmoid()
    
    def forward(self, x, y):
        pos = x[:len(x)//2]
        neg = x[len(x)//2:]
        return -torch.mean(self.m((pos-neg)))
    

In [47]:
"""
Calculate F1/MAP score
"""

import numpy as np
import random

def calc_score(dataset, model):
    with torch.no_grad():
        matrix = torch.matmul(model.user_matrix, model.item_matrix).cpu().detach().numpy()
        sum_AP = 0
        for i, row in enumerate(matrix):
            if dataset.is_train:
                golden_set = set(dataset.train_lists[i])
            else:
                golden_set = set(dataset.valid_lists[i])
                row[dataset.train_lists[i]] = np.min(row)
            rank = np.flip(np.argsort(row))
            n_recom = 50
            AP = 0
            true_pos = 0
            # use all pos (including test data) to calculate MAP (?)
            n_pos = int((len(dataset.train_lists[i]) + len(dataset.valid_lists[i]))/0.89)
            for i, idx in enumerate(rank[:n_recom]):
                if idx in golden_set:
                    true_pos += 1
                    AP += (true_pos/(i+1))/n_pos
            sum_AP += AP
    return sum_AP/len(dataset)
"""
torch.manual_seed(66)
random.seed(66)
dataset = PIR_Dataset(train_file, 0.1)
dataset.is_train = True
user_file = "../model/user0.196863.pt"
item_file = "../model/item0.196863.pt"
model = PIR_Model(dataset.num_users, dataset.num_items)
model.user_matrix = torch.load(user_file)
model.item_matrix = torch.load(item_file)
print(calc_score(dataset, model))
""""""

0.3304835278205502


In [24]:
"""
PIR_Trainer
"""

import torch.optim as optim
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm
import random
import os

ITER = 1000
torch.manual_seed(66)
random.seed(66)
checkpoint_dir = "../model/"
dataset = PIR_Dataset(train_file, 0.1)
model = PIR_Model(dataset.num_users, dataset.num_items, 64)
loss_func = BPR_Loss()
# loss_func = BCEWithLogitsLoss()
optimizer = optim.Adam([model.user_matrix, model.item_matrix], lr=1e-3, weight_decay=1e-6)

prev_valid_loss = 100000
valid_tolerance = 3
idxs = [i for i in range(len(dataset))]


for i in range(ITER):
    if valid_tolerance == 0:
        break
    train_loss = 0
    valid_loss = 0
    random.shuffle(idxs)
    dataset.set_is_train(True)
    print("training...")
    for uid in tqdm(idxs):
        optimizer.zero_grad()
        s1, s2, target = dataset[uid]
        output = model(uid, s1, s2)
        loss = loss_func(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    train_MAP = calc_score(dataset, model)
    print("validating...")
    dataset.set_is_train(False)
    for uid in tqdm(idxs):
        with torch.no_grad():
            s1, s2, target = dataset[uid]
            output = model(uid, s1, s2)
            loss = loss_func(output, target)
            valid_loss += loss.item()
    valid_MAP = calc_score(dataset, model)
    print("iter {}: training loss={:6.4f}, MAP={:6.4f}; validation loss={:6.4f}, MAP={:6.4f}\n" \
          .format(i, train_loss/len(idxs), train_MAP, valid_loss/len(idxs), valid_MAP))
    if valid_loss < prev_valid_loss:
        if prev_valid_loss != 100000:
            os.remove("{}user{:8.6f}.pt".format(checkpoint_dir, prev_valid_loss/len(idxs)))
            os.remove("{}item{:8.6f}.pt".format(checkpoint_dir, prev_valid_loss/len(idxs)))
        torch.save(model.user_matrix, "{}user{:8.6f}.pt".format(checkpoint_dir, valid_loss/len(idxs)))
        torch.save(model.item_matrix, "{}item{:8.6f}.pt".format(checkpoint_dir, valid_loss/len(idxs)))
        prev_valid_loss = valid_loss
        valid_tolerance = 3
    else:
        valid_tolerance -= 1


training...


100%|█████████████████████████████████████████| 4454/4454 [00:32<00:00, 136.41it/s]


validating...


100%|█████████████████████████████████████████| 4454/4454 [00:07<00:00, 581.04it/s]


iter 0: training loss=1.6217548194812668, validation loss=1.5775860477620205

training...


...

iter 41: training loss=0.09504774448553416, validation loss=0.2093686984077013

training...


100%|█████████████████████████████████████████| 4454/4454 [00:36<00:00, 122.84it/s]


validating...


100%|█████████████████████████████████████████| 4454/4454 [00:08<00:00, 532.90it/s]


iter 42: training loss=0.09445019559857369, validation loss=0.20847767490614091



In [48]:
"""
Evaluate and Test
"""

import numpy as np

output_file = "../outputs/out.csv"
user_file = ["../model/user0.196863.pt", "../model/user0.199941.pt"]
item_file = ["../model/item0.196863.pt", "../model/item0.199941.pt"]
model = PIR_Model(dataset.num_users, dataset.num_items)
matrix = np.zeros((dataset.num_users, dataset.num_items))
for f1, f2 in zip(user_file, item_file):
    matrix += torch.matmul(torch.load(f1), torch.load(f2)).cpu().detach().numpy()
sum_true_pos = 0
sum_poss_pos = 0

with open(output_file, "w") as f:
    f.write("UserId,ItemId\n")
#     matrix = torch.matmul(model.user_matrix, model.item_matrix).cpu().detach().numpy()
    for i, row in enumerate(matrix):
        f.write("{},".format(i))
        rank = np.flip(np.argsort(row))
        pos_set = set(dataset.train_lists[i]+dataset.valid_lists[i])
        n_recom = 50
        sum_poss_pos += n_recom
        for idx in rank:
            if n_recom == 0:
                break
            if idx not in pos_set:
                n_recom -= 1
                f.write("{} ".format(idx))
            else:
                sum_true_pos += 1
                
        f.write("\n")            
print(sum_true_pos, sum_poss_pos)

180570 222700


In [ ]:
"""
Test optimizing ability after indexing the tensor
"""

import torch
import torch.nn as nn
import torch.optim as optim

loss = nn.BCEWithLogitsLoss()
input_prev = torch.tensor([[1.0,1.0,0.0,0.0,1.0,1.0], [1.0,1.0,0.0,0.0,1.0,1.0]], requires_grad=True)
optimizer = optim.Adam([input_prev], lr=0.0001)

input = input_prev[[[0,0,0], [0,3,4]]]
target = torch.empty(3).random_(2)

print(input_prev)
print(input)
print(target)

output = loss(input, target)
output.backward()
optimizer.step()

print(input_prev)
print(input)
print(target)
print(output)